In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import pandas as pd

df_trips = pd.read_csv('data_bus/trips.txt')
df_trips = df_trips.dropna(how='all', axis=1)

df_stop_times = pd.read_csv('data_bus/stop_times.txt')
df_stop_times = df_stop_times.dropna(how='all', axis=1)

df_stops = pd.read_csv('data_bus/stops.txt')
df_stops = df_stops.dropna(how='all', axis=1)

df_routes = pd.read_csv('data_bus/routes.txt')
df_routes = df_routes.dropna(how='all', axis=1)

df_shapes = pd.read_csv('data_bus/shapes.txt')
df_shapes = df_shapes.dropna(how='all', axis=1)


In [2]:
'''
＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝
データ処理
＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝
'''

df_route = pd.merge(df_trips, df_routes, on='route_id') 
df_bus = df_route.drop_duplicates(subset='trip_id')

df_bus_stop = pd.merge(df_stop_times, df_stops, on='stop_id')

sr_shape_id = df_shapes.shape_id.unique()
df_all_routes = pd.DataFrame(index=[], columns=['shape_id', 'shape'])

for shape_id in sr_shape_id:
    df_shape = df_shapes[df_shapes['shape_id']==shape_id]
    tpl_shape = tuple(zip(df_shape.loc[:,'shape_pt_lon'], df_shape.loc[:,'shape_pt_lat']))
    sr_shape = pd.Series([shape_id, tpl_shape], index=df_all_routes.columns)
    df_all_routes = df_all_routes.append(sr_shape, ignore_index=True)


In [3]:
from google.transit import gtfs_realtime_pb2 # pip install --upgrade gtfs-realtime-bindings
import requests
from retry import retry

'''
ＧＴＦＳ（動的データ）の取得
'''
COLS = ['trip_id','vehicle','timestamp','stop_sq','status','latitude','longitude']

@retry(tries=3, delay=2, backoff=2)
def get_realtime_data():
    feed = gtfs_realtime_pb2.FeedMessage()
    response = requests.get('http://opendata.sagabus.info/vehicle.pb')
    feed.ParseFromString(response.content)

    df_result = pd.DataFrame(columns=COLS)

    for entity in feed.entity:
        if entity.HasField('vehicle'):
            sr_data = pd.Series([
                entity.vehicle.trip.trip_id,
                entity.vehicle.vehicle.id,
                entity.vehicle.timestamp,
                entity.vehicle.current_stop_sequence,
                entity.vehicle.current_status,
                entity.vehicle.position.latitude,
                entity.vehicle.position.longitude 
            ], index=df_result.columns)
            df_result = df_result.append(sr_data, ignore_index=True)          
    return df_result

In [4]:
import folium
from IPython.display import IFrame

bus_map = folium.Map(location=[33.25, 130.3], zoom_start=13)

df_vehicle = get_realtime_data()
df_vehicle = pd.merge(df_vehicle, df_bus, on='trip_id')


'''
＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝
見える化
＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝
第1層　全線路図
'''

def style_1st_function(feature):
    return {
        'fillOpacity': 1,
        'weight': 2,
        'color': 'blue'
    }

first_layer = folium.FeatureGroup(name='すべての路線')

for tpl_shape in df_all_routes.itertuples():
    line_1st = folium.GeoJson(
        data={
            'type': 'LineString',
            'coordinates': tpl_shape.shape
        },
        overlay=True,
        style_function=style_1st_function
    )
    line_1st.add_to(first_layer)
first_layer.add_to(bus_map)


'''
第2層　運行中バスの線路図
'''

def style_2nd_function(feature):
    return {
        'fillOpacity': 1,
        'weight': 4,
        'color': 'red'
    }

second_layer = folium.FeatureGroup(name='運行中の路線')

for bus in df_vehicle.itertuples():
    shape_id = df_route[df_route['trip_id']==bus.trip_id]['shape_id']
    route = df_all_routes[df_all_routes['shape_id']==shape_id.values[0]]
    line_2nd = folium.GeoJson(
        data={
            'type': 'LineString',
            'coordinates': route['shape'].values[0]
        },
        overlay=True,
        style_function=style_2nd_function
    )
    line_2nd.add_to(second_layer)
second_layer.add_to(bus_map)


'''
第3層　運行中バス
'''
third_layer = folium.FeatureGroup(name='運行中のバス')

for bus in df_vehicle.itertuples():
    if bus.status == 1:
        color = 'blue'
    elif bus.status == 2:
        color = 'red'
    else:
        color = 'gray'

    folium.Marker(
        location = [bus.latitude, bus.longitude], 
        icon=folium.Icon(color=color, icon='bus', prefix='fa')
    ).add_to(third_layer)
    
third_layer.add_to(bus_map)
folium.LayerControl(collapsed=False).add_to(bus_map)


# HTML出力
html = 'result/real_time_bus_map.html'
bus_map.save(html)
IFrame(html, width=750, height=750)
